In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score, mean_squared_error
from dmba import stepwise_selection
from dmba import AIC_score

In [44]:
insurance=pd.read_csv(r'C:\Users\hp RED\AIfundamentals\Datos\insurance.csv',delimiter=',')
insurance

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [45]:
#Información de mi base de datos
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [46]:
#Revisión de espacios vacíos
insurance.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
#duplicates = []
#for col in range(insurance.shape[1]):
 #   contents = insurance.iloc[:, col]
    
#    for comp in range(col + 1, insurance.shape[1]):
#        if contents.equals(insurance.iloc[:, comp]):
 #           duplicates.append(comp)
#duplicates = np.unique(duplicates).tolist()
#duplicates

In [47]:
insurance=pd.get_dummies(insurance,columns=['sex'])
insurance

,age,bmi,children,smoker,region,charges,sex_female,sex_male
0,19,27.900,0,yes,southwest,16884.92400,1,0
1,18,33.770,1,no,southeast,1725.55230,0,1
2,28,33.000,3,no,southeast,4449.46200,0,1
3,33,22.705,0,no,northwest,21984.47061,0,1
4,32,28.880,0,no,northwest,3866.85520,0,1
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,no,northwest,10600.54830,0,1
1334,18,31.920,0,no,northeast,2205.98080,1,0
1335,18,36.850,0,no,southeast,1629.83350,1,0
1336,21,25.800,0,no,southwest,2007.94500,1,0


In [48]:
insurance=pd.get_dummies(insurance,columns=['smoker'])
insurance

,age,bmi,children,region,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,27.900,0,southwest,16884.92400,1,0,0,1
1,18,33.770,1,southeast,1725.55230,0,1,1,0
2,28,33.000,3,southeast,4449.46200,0,1,1,0
3,33,22.705,0,northwest,21984.47061,0,1,1,0
4,32,28.880,0,northwest,3866.85520,0,1,1,0
...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,northwest,10600.54830,0,1,1,0
1334,18,31.920,0,northeast,2205.98080,1,0,1,0
1335,18,36.850,0,southeast,1629.83350,1,0,1,0
1336,21,25.800,0,southwest,2007.94500,1,0,1,0


In [49]:
insurance=pd.get_dummies(insurance,columns=['region'])
insurance

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [50]:
data=insurance
data

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1,0,0,1,0,0,0,1
1,18,33.770,1,1725.55230,0,1,1,0,0,0,1,0
2,28,33.000,3,4449.46200,0,1,1,0,0,0,1,0
3,33,22.705,0,21984.47061,0,1,1,0,0,1,0,0
4,32,28.880,0,3866.85520,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,0,1,1,0,0,1,0,0
1334,18,31.920,0,2205.98080,1,0,1,0,1,0,0,0
1335,18,36.850,0,1629.83350,1,0,1,0,0,0,1,0
1336,21,25.800,0,2007.94500,1,0,1,0,0,0,0,1


In [51]:
predictors=['age','bmi','children','sex_female','sex_male','smoker_no','smoker_yes',
            'region_northeast','region_northwest','region_southeast','region_southwest']
outcome=['charges']
insurance_lm=LinearRegression()
insurance_lm.fit(data[predictors],data[outcome])
insurance_lm

LinearRegression()

In [52]:
print('Intercept',insurance_lm.intercept_)
for name, coef in zip(predictors,insurance_lm.coef_):
    print(f'{name}:{coef}')

Intercept [-666.93771994]
age:[   256.85635254    339.19345361    475.50054515     65.6571797
    -65.6571797  -11924.26727096  11924.26727096    587.00923503
    234.0453356    -448.01281436   -373.04175627]


In [29]:
model=sm.OLS(data[outcome],data[predictors].assign(const=1))
results=model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.749
Method:                 Least Squares   F-statistic:                     500.8
Date:                Mon, 23 May 2022   Prob (F-statistic):               0.00
Time:                        23:39:09   Log-Likelihood:                -13548.
No. Observations:                1338   AIC:                         2.711e+04
Df Residuals:                    1329   BIC:                         2.716e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
age                256.8564     11.899     21.587      0.000     233.514     280.199
bmi                339.1935     28.599     11.860      0.000     283.088     395.298
children           475.5005    137.804      3.451      0.001     205.163     745.838
sex_female         -82.5512    269.226     -0.307      0.759    -610.706     445.604
sex_male          -213.8656    274.976     -0.778      0.437    -753.299     325.568
smoker_no        -1.207e+04    282.338    -42.759      0.000   -1.26e+04   -1.15e+04
smoker_yes        1.178e+04    313.530     37.560      0.000    1.12e+04    1.24e+04
region_northeast   512.9050    300.348      1.708      0.088     -76.303    1102.113
region_northwest   159.9411    301.334      0.531      0.596    -431.201     751.083
region_southeast  -522.1170    330.759     -1.579      0.115   -1170.983     126.749
region_southwest  -447.1459    310.933     -1.438      0.151   -1057.119     162.827
const             -296.4168    430.507     -0.689      0.491   -1140.964     548.130
==============================================================================
Omnibus:                      300.366   Durbin-Watson:                   2.088
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              718.887
Skew:                           1.211   Prob(JB):                    7.86e-157
Kurtosis:                       5.651   Cond. No.                     5.17e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.32e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [30]:
rows=len(data.index)
fitted = insurance_lm.predict(data[predictors])
RMSE = np.sqrt(mean_squared_error(data[outcome], fitted))
r2 = r2_score(data[outcome], fitted)
print(f'RMSE: {RMSE:.0f}')
print(f'r2: {r2:.4f}')
print(rows)
adj_r2=1-(1-r2)*((rows-1)/(rows-len(predictors)-1))
print(f'adj_r2:{adj_r2:.4f}')

RMSE: 6042
r2: 0.7509
1338
adj_r2:0.7488


In [33]:
#stepwise selection


y=data[outcome]
X=data[predictors]

def train_model(variables):
    if len(variables)==0:
        return None
    model=LinearRegression()
    model.fit(X[variables],y)
    return model

def score_model(model,variables):
    if len(variables)==0:
        return AIC_score(y,[y.mean()]*len(y),model,df=1)
    return AIC_score(y,model.predict(X[variables]),model)

best_model,best_variables= stepwise_selection(X.columns, train_model,score_model,verbose=True)
print(best_model.intercept_)
print('Coefficients')
for name, coef in zip(best_variables,best_model.coef_):
    print(name,coef)

Variables: age, bmi, children, sex_female, sex_male, smoker_no, smoker_yes, region_northeast, region_northwest, region_southeast, region_southwest
Start: score=28959.26, constant
Step: score=27667.46, add smoker_no
Step: score=27251.32, add age
Step: score=27119.84, add bmi
Step: score=27108.04, add children
Step: score=27104.10, add region_northeast
Step: score=27101.69, add region_northwest
Step: score=27101.53, add sex_female
Step: score=27101.51, add region_southeast
Step: score=27101.51, unchanged None
[10818.63061505]
Coefficients
smoker_no [-23848.53454191    256.85635254    339.19345361    475.50054515
    960.0509913     607.08709188    131.3143594     -74.97105809]


In [ ]:
predictors=['smoker_no','age','bmi','children','region_northeast','region_northwest','sex_female','region_southeast']
outcome=['']